# Supplement Sales Prediction


Your Client WOMart is a leading nutrition and supplement retail chain that offers a comprehensive range of products for all your wellness and fitness needs. 

WOMart follows a multi-channel distribution strategy with 350+ retail stores spread across 100+ cities. 

Effective forecasting for store sales gives essential insight into upcoming cash flow, meaning WOMart can more accurately plan the cashflow at the store level.

Sales data for 18 months from 365 stores of WOMart is available along with information on Store Type, Location Type for each store, Region Code for every store, Discount provided by the store on every day, Number of Orders everyday etc.

The Task is to predict the store sales for each store in the test set for the next two months.

# 1. Data Preparation


The dataset contain the two files in the csv format:

TRAIN.csv : this file has 188340 rows and 10 columns.

TEST_FINAL.csv : this file has 22265 rows and 8 columns.

Variable : Definition

ID : Unique Identifier for a row

Store_id : Unique id for each Store

Store_Type : Type of the Store

Location_Type : Type of the location where Store is located

Region_Code :Code of the Region where Store is located

Date : Information about the Date

Holiday : If there is holiday on the given Date, 1 : Yes, 0 : No

Discount : If discount is offered by store on the given Date, Yes/ No

Orders : Number of Orders received by the Store on the given Day
    
Sales : Total Sale for the Store on the given Day

Importing Neccessary libraries

In [ ]:
import pandas as pd
import numpy as sns
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
%matplotlib inline

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)

Loading csv files to dataframe

In [ ]:
df_train= pd.read_csv('TRAIN.csv')
df_test = pd.read_csv('TEST_FINAL.csv')

checking for missing values and data types in training set

In [ ]:
df_train.info()

In [ ]:
df_train.isnull().sum()

- No missing values.Data set looks clean
- Date need to be converted into Date time instead of object

Checking for missing values and data types in test data set

In [ ]:
df_test.info()

In [ ]:
df_test.isnull().sum()

- NO missing values found and data set look clean. order catergory is not in test data set
-Date need to be converted into Date time instead of object

First look of the train data set

In [ ]:
df_train.head()

First look of the test data set

In [ ]:
df_test.head()

Before we change anything in the original Dataframe, we create a copy

In [ ]:
EDA_train = df_train.copy()
EDA_test = df_test.copy()

# 2. Exploratory Data Analysis

Before we starting to explority data analysis we need to convert Date column object as  Date time

In [ ]:
EDA_train["Date"]=  pd.to_datetime(EDA_train["Date"])

In [ ]:
EDA_train.head()

In [ ]:
EDA_train.info()

The Date object converted to Date time.Repeating the same step for test data set

In [ ]:
EDA_test["Date"]=  pd.to_datetime(EDA_test["Date"])

In [ ]:
EDA_test.info()

In [ ]:
EDA_test.head()

In [ ]:
EDA_train['Sales'].describe()

Judging by the mean and maximum values of sales, there are some outliers.
We need to keep this in mind and will exclude them for our first visualisations

Question: How many shops and products do we have? 

In [ ]:
EDA_train.nunique()

-We have 365 stores in train data set, 4 store types distributed among in 5 location types with 4 Region codes

In [ ]:
EDA_test.nunique()

-Need to check any corelation between #orders and sales

In [ ]:
correlation = EDA_train['#Order'].corr(EDA_train['Sales'])
print("Correlation between oder and sales:",correlation)
sns.scatterplot(x=EDA_train['#Order'],y=EDA_train['Sales'])

There a postive correlation between sales and orders .so only considering sales figure from onwards for Exploratorty Data Analysis

-We have 365 stores in test data set, 4 store types distributed among in 5 location types with 4 Region codes

What is our target?
-We want to predict the number of items sold for test data .
Let's go ahead and take a closer look at the items.


Finding how many stores for each category of store types

## Store Types

In [ ]:
typecounts = EDA_train.Store_Type.value_counts().to_dict()
EDA_train_store_types = pd.DataFrame(list(typecounts.items()), columns=['Store_Type', 'Counts'])
EDA_train_store_types.sort_values(by='Counts',inplace=True,ascending=False)
EDA_train_store_types

In [ ]:
#plotting figure of each store types as pie chart

In [ ]:
fig = px.pie(EDA_train_store_types, values='Counts', names='Store_Type',
             title='Popularity of Store Types',labels='Store_Type')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

- S1 have large number of store counts,then S4
- Lets find out the average sales of Each sales by box plot

In [ ]:

EDA_train.boxplot(by ='Store_Type', column =['Sales'])

- it is shown s4 have more average sales than s1
- having a lagre number og outliers.need to check these Outliers have any significance in data
- The number of stores doesn't have much importance in sales
- to verify this let's check it with order number by plotting box plot


In [ ]:
EDA_train.boxplot(by ='Store_Type', column =['#Order'])

- As previsoly mentioned the orders and sales have a strong correlation the sales and orders show same type of plots

Finding average sales of Each stores

In [ ]:
avg_sales = EDA_train.groupby('Store_Type')['Sales'].mean().to_dict()
EDA_Train_avg_sales_storetype = pd.DataFrame(list(avg_sales.items()), columns=['Store_Type', 'AvgSales'])

EDA_Train_avg_sales_storetype 

In [ ]:
fig = px.bar(EDA_Train_avg_sales_storetype, 
             x="Store_Type", 
             y="AvgSales",  
             title="Averge Sales - Per Store",
             color_discrete_sequence=["#DC143C"], template='plotly_dark')
fig.show()


- s4 and s3 have high average sales

## EDA on Location types

In [ ]:
location_counts = EDA_train.Location_Type.value_counts().to_dict()
EDA_train_location_counts = pd.DataFrame(list(location_counts.items()), columns=['Location_Type', 'Counts'])


In [ ]:
fig = px.pie(EDA_train_location_counts, values='Counts', names='Location_Type',
             title='Popularity of location Types',labels='location_Type')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

- L1 and L2 location have High number of stores

In [ ]:
EDA_train.boxplot(by ='Location_Type', column =['Sales'])

In [ ]:
avg_sales_Location = EDA_train.groupby('Location_Type')['Sales'].mean().to_dict()
EDA_Train_avg_sales_Locationtype = pd.DataFrame(list(avg_sales_Location.items()), columns=['Location_Type', 'AvgSales'])

EDA_Train_avg_sales_Locationtype

In [ ]:
fig = px.bar(EDA_Train_avg_sales_Locationtype, 
             x="Location_Type", 
             y="AvgSales",  
             title="Averge Sales - Per Location",
             color_discrete_sequence=["#DC143C"], template='plotly_dark')
fig.show()

- Location Types also effect the average sales 

# EDA on Regions types

In [ ]:
region_counts = EDA_train.Region_Code.value_counts().to_dict()
EDA_train_region_counts = pd.DataFrame(list(region_counts.items()), columns=['Region_Type', 'Counts'])

fig = px.pie(EDA_train_region_counts, values='Counts', names='Region_Type',
             title='Popularity of Region Types',labels='Region_Type')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
EDA_train.boxplot(by ='Region_Code', column =['Sales'])

In [ ]:
avg_sales_Region= EDA_train.groupby('Region_Code')['Sales'].mean().to_dict()
EDA_Train_avg_sales_Region_code = pd.DataFrame(list(avg_sales_Region.items()), columns=['Region_Code', 'AvgSales'])

EDA_Train_avg_sales_Region_code

In [ ]:
fig = px.bar(EDA_Train_avg_sales_Region_code, 
             x="Region_Code", 
             y="AvgSales",  
             title="Averge Sales - Per Regions",
             color_discrete_sequence=["#DC143C"], template='plotly_dark')
fig.show()

- Regions also a dependent feature on sales numbers

# EDA on Discount

To find How Giving Discount Effect

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Discount_sales = EDA_train.groupby('Discount')['Sales'].mean()
Discount_counts = EDA_train.Holiday.value_counts()

print(Discount_sales)
print(Discount_counts)

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Mean Discount Sales Amount", "Discount Given Days"))

fig.add_trace(go.Bar(x=Discount_sales.values, y=Discount_sales.index, orientation='h',),1, 1)

fig.add_trace(go.Bar(x=Discount_sales.values, y=Discount_sales.index, orientation='h',),1, 2)

fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), template='plotly_dark', showlegend=False)
fig.show()

- Discounted Days gives higher number of sales

plotting line plot to find the same

In [ ]:
sns.lineplot(data=EDA_train, x="Date", y=(EDA_train['Sales']),hue='Discount')

- Discount given days have higher sales figure

# EDA on Holiday

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

holiday_sales = EDA_train.groupby('Holiday')['Sales'].mean()
holiday_counts = EDA_train.Holiday.value_counts()

print(holiday_sales)
print(holiday_counts)

In [ ]:
holiday_sales.dtypes

In [ ]:
EDA_Train_avg_sales_holidays = pd.DataFrame(list(holiday_sales.items()), columns=['Holidays', 'AvgSales'])

EDA_Train_avg_sales_holidays

In [ ]:
EDA_Train_avg_sales_holidays['Holidays'] = EDA_Train_avg_sales_holidays['Holidays'].map(
                   {True:'Yes',False:'No'})

In [ ]:
EDA_Train_avg_sales_holidays

In [ ]:
EDA_Train_holidays_count = pd.DataFrame(list(holiday_counts.items()), columns=['Holidays', 'No of Hoildays'])

EDA_Train_holidays_count

In [ ]:
EDA_Train_holidays_count['Holidays'] =EDA_Train_holidays_count['Holidays'].map(
                   {True:'Yes' ,False:'No'})
EDA_Train_holidays_count

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Holidays/Nonholidays Sales", "Holidays/Nonholidays Counts"))

fig.add_trace(go.Bar(x=EDA_Train_avg_sales_holidays['AvgSales'], y=EDA_Train_avg_sales_holidays['Holidays'], orientation='h',),1, 1)

fig.add_trace(go.Bar(x=EDA_Train_holidays_count['No of Hoildays'], y=EDA_Train_holidays_count['Holidays'], orientation='h',),1, 2)

fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), template='plotly_dark', showlegend=False)
fig.show()

- HOlidays have also having sales number.

plotting line plot of mean average sales with hue as Hoilday 

In [ ]:
sns.lineplot(data=EDA_train, x="Date", y=(EDA_train['Sales']),hue='Holiday')

- Hoilday have higher effect in sales number
- some hoildays have steep decline in sales numbers.need to find out how manys days have sales below a threshold value

# Checking Outliers in sales figure

setting a threshold as 20000 as sales figure..exploring values below

In [ ]:
EDA_train[EDA_train['Sales'] <= 20000 ]

ploting the sales figure hue holiday

In [ ]:
sns.lineplot(data=EDA_train[EDA_train['Sales'] < 20000 ], x="Date", y=(EDA_train['Sales']),hue='Holiday')

ploting the sales figure hue Discount

In [ ]:
sns.lineplot(data=EDA_train[EDA_train['Sales'] < 20000 ], x="Date", y=(EDA_train['Sales']),hue='Discount')

By checking both the figure it is found that there are some outliers in data which gives low sales in some days.

 checking data for zero sales

In [ ]:
EDA_train[EDA_train['Sales'] == 0]

some days only giving zero sales.later decide whether data need to removed or not.

# Insights from EDA


- No missing values.Data set looks clean
- There a postive correlation between sales and orders.so droping order doesn't need to consider while traning(Also order numbers are not in test set)
- ID can be removed since there is no relation between sales 
- Location type,Region,Store type have different sales figure.considerd for training
- Holiday and Discounthave effect in sales number.Considered for training
- Removing outliers in sales number considerd after training the model if desired accuaracy is not achieved


# 3.Data Preparation for Model Training

Following steps will be performed for preparing the data for the subsequent model training

- Based on the Exploratory Data Analysis and Coorelation study, the columns with weak relationship with the target column will be dropped
- Input and Target dataframes will be created
- Need to split datetime object into day,year,month for both test and train
- Label encoding methods applied to categorical columns
- Training and Validation datasets will be created
- A function will be defined based on which the models performance will be measured

In [ ]:
EDA_train.head()

In [ ]:
EDA_test.head()

- Creating copy of both train and test set for Data prepration and model training

In [ ]:
FE_train = EDA_train.copy()

In [ ]:
FE_test = EDA_test.copy()

In [ ]:
FE_train.head()

In [ ]:
FE_test.head()

 - creating new column Day,month and year

In [ ]:
FE_train['Year'] = FE_train['Date'].dt.year
FE_train['Day'] = FE_train['Date'].dt.day
FE_train['Month'] = FE_train['Date'].dt.month


In [ ]:
FE_train.head()

Droping ID,Date,'#Order' for train test set

In [ ]:
FE_train_2 = FE_train.drop(['#Order','Date'],axis=1)

In [ ]:
FE_train_2.head()

- Checking data types of FE_train_2

In [ ]:
FE_train_2.dtypes

Repeat the same step for test set

In [ ]:
FE_test['Year'] = FE_test['Date'].dt.year
FE_test['Day'] = FE_test['Date'].dt.day
FE_test['Month'] = FE_test['Date'].dt.month

In [ ]:
FE_test.head()

- Droping ID,Date for test set

In [ ]:
FE_test_2 = FE_test.drop(['Date'],axis=1)

In [ ]:
FE_test_2.head()

In [ ]:
FE_test_2.dtypes

# 3.1 Encoding

In [ ]:
from xgboost import XGBRegressor
from sklearn import ensemble
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline


In [ ]:
encoder = LabelEncoder()

- creating copy of test and train dataset for Encoding

In [ ]:
EC_train = FE_train_2.copy()

In [ ]:
EC_train.columns

In [ ]:
EC_train.head()

In [ ]:
EC_test = FE_test_2.copy()

In [ ]:
EC_test.head()

In [ ]:
for i in EC_train.columns: 
    if EC_train[i].dtype == 'object': 
        encoder.fit_transform(list(EC_train[i].values)) #To Fit transform
        EC_train[i] = encoder.transform(EC_train[i].values) # TO fit Transform
         
        for j in EC_train.columns: 
            if EC_train[j].dtype == 'int':
                EC_train[j] = EC_train[j].astype('float64') #To Change the type

for k in EC_test.columns: 
    if EC_test[k].dtype == 'object': 
        encoder.fit_transform(list(EC_test[k].values)) #To Fit transform
        EC_test[k] = encoder.transform(EC_test[k].values) #FTO Transform
         
        for m in EC_test.columns: 
            if EC_test[m].dtype == 'int':
                EC_test[m] = EC_test[m].astype('float64')

In [ ]:
EC_test.head()

- Splitting X and y

In [ ]:
X = EC_train.drop(columns=['ID', 'Sales']) # Data X
y = EC_train['Sales'] # Data y     
X_test = EC_test.drop(columns=['ID'])

In [ ]:
X_train, X_cv, y_train, y_cv = train_test_split(
    X, y,test_size=0.3, 
    random_state=42)

# 3.2 PipelIne and training Data

In [ ]:
pipeline_XGB = Pipeline([ # Our Pipeline
    ('scaler', MinMaxScaler()),
    ('transformer', QuantileTransformer()),
    ('model', XGBRegressor(
        learning_rate=0.2,
        n_estimators=10000,
        random_state=42,
        objective='reg:squarederror',
        booster='gbtree'
    ))
])

pipeline_XGB.fit(X_train, y_train) # Train Data

In [ ]:
from sklearn import metrics
import math

In [ ]:
XGB_pred_train = pipeline_XGB.predict(X_train) # Predict Train Data
XGB_pred_cv = pipeline_XGB.predict(X_cv) # Predict Valid Data

In [ ]:
Train_r2_score = metrics.r2_score(y_train, XGB_pred_train) # R2_score
print(f'Train R2_score: {Train_r2_score }')

Train_mse = metrics.mean_squared_error(y_train, XGB_pred_train) # MSE Score
print(f'Train MSE : {Train_mse}')

Train_RMSE = math.sqrt(metrics.mean_squared_error(y_train,XGB_pred_train)) # SQRT MSE Score
print(f'Train RMSE : {Train_RMSE}')

In [ ]:
train = pd.DataFrame(
    {'Predicted Sales':XGB_pred_train, 'Actual Sales':y_train}
)

fig= plt.figure(
    figsize=(16, 9)
)

train = train.reset_index()
train = train.drop(
    ['index'],axis=1
)

plt.plot(train[:50])
plt.legend(['Actual Sales','Predicted Sales'])
plt.title('Actual & Predicted Sales')
plt.show()

# Evaluate Test Data

In [ ]:
Test_r2_score = metrics.r2_score(y_cv, XGB_pred_cv) # R2_score
print(f'Test R2_score: {Test_r2_score}')

Test_mse = metrics.mean_squared_error(y_cv, XGB_pred_cv) # MSE Score
print(f'Test MSE : {Test_mse}')

Test_RMSE = math.sqrt(metrics.mean_squared_error(y_cv, XGB_pred_cv)) # SQRT MSE Score
print(f'Test RMSE : {Test_RMSE}')

In [ ]:
test = pd.DataFrame(
    {'Predicted Sales':XGB_pred_cv, 'Actual Sales':y_cv}
)

fig= plt.figure(
    figsize=(16, 9)
)

test = test.reset_index()
test = test.drop(
    ['index'],axis=1
)

plt.plot(test[:50])
plt.legend(['Actual Sales','Predicted Sales'])
plt.title('Actual & Predicted Sales')
plt.show()

In [ ]:
Test_prediction = pipeline_XGB.predict(X_test)

In [ ]:
Test_prediction

In [ ]:
Test_data = pd.read_csv('SAMPLE.csv')

In [ ]:
Test_data.head()

In [ ]:
FE_test_2.head()

In [ ]:
Test_3= FE_test_2.filter(['ID'],axis=1)

In [ ]:
Test_3.head()

In [ ]:
Test_3['Sales'] = Test_prediction

In [ ]:
Test_3.head()

In [ ]:
Test_3.to_csv('XGB.csv',index=True)